In [10]:
import pandas as pd
import soynlp
import string

In [11]:
# 데이터 읽기

file_path = 'naver_comments.csv'
corpus_path = 'naver_cmt_dic.csv'

colnames = ['a', 'b', 'c', 'd']
df = pd.read_csv(file_path, encoding='utf-8', header=None, names=colnames)
# df = pd.read_excel(file_path, encoding='utf-8', names=colnames)
df['b'] = df['b'].dropna().apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
df.iloc[:, 1:2].to_csv(corpus_path, encoding='utf-8', index=False)

In [12]:
len(df)

334633

In [13]:
pd.read_csv(corpus_path, encoding='utf-8').head()

,b
0,연대 법대출신 토익925점 kt취업vs건대 충주캠 무스펙 귀걸이낀 증명사진 이력서 ...
1,전직 공안 검사 출신 이자나없는 일도 만들고 없던 말도 만들고그렇게 일반인 빵갱이 ...
2,교활이 이새키는 503호뇬 밑에 있을때는 다들 벙어리인줄 알았어헐 근데 지금보니...
3,이렇게 입이 가벼운 작자가 어찌 대권을 노린다는건지 한심하지 짝이 없는 개누리당
4,연세대나온 아들이 KT간게 뭐대수라고 지잡나온 귀걸이 아빠나 조사해봐라 정권의 개들아


In [14]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

In [15]:
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
noun_extractor.train(sents)
nouns = noun_extractor.train_extract(sents, min_noun_frequency=335,
                                     min_noun_score=0.6)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 939421 from 547626 sents. mem=1.262 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4403996, mem=2.680 Gb
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 939421 from 547626 sents. mem=2.740 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4403996, mem=3.543 Gb
[Noun Extractor] batch prediction was completed for 342463 words
[Noun Extractor] checked compounds. discovered 257349 compounds
[Noun Extractor] postprocessing detaching_features : 1289 -> 1273
[Noun Extractor] postprocessing ignore_features : 1273 -> 1214
[Noun Extractor] postprocessing ignore_NJ : 1214 -> 1152
[Noun Extractor] 1152 nouns (257349 compounds) with min frequency=335
[Noun Extractor] flushing was done. 

In [16]:
a = [noun for noun, score in nouns.items() if len(noun) > 1]
a = [i.replace('"', '') for i in a] 
a = [i.replace("'", '') for i in a] 
len(a)

1045

In [17]:
a = pd.DataFrame(a, columns=['word'])
a['pos'] = 'NNP'
a = a[a['word'].str[-1] != '들']
# a = a[a['word'].str[-1] != '님']
a = a.drop_duplicates()
len(a)

991

In [18]:
a.to_csv(corpus_path, encoding='utf-8', index=False, header=False)